In [7]:
# Importing necessary modules
from random import randint
from decimal import Decimal
from fhir.resources.R4B.condition import Condition
from fhir.resources.R4B.condition import fhirtypes
from fhir.resources.R4B.identifier import Identifier
from fhir.resources.R4B.codeableconcept import CodeableConcept
from fhir.resources.R4B.coding import Coding
from fhir.resources.R4B.reference import Reference
from datetime import datetime, timezone

ConditionCategory_Dict= {
    "Problem List Item": {
        "system": "http://terminology.hl7.org/CodeSystem/condition-category",
        "code": "problem-list-item",
        "display": "Problem List Item"
    },
    "Encounter Diagnosis": {
        "system": "http://terminology.hl7.org/CodeSystem/condition-category",
        "code": "encounter-diagnosis",
        "display": "Encounter Diagnosis"
    }
}

ConditionClinicalStatus_Dict= {
    "Active": {
        "system": "http://terminology.hl7.org/CodeSystem/condition-clinical",
        "code": "active",
        "display": "Active"
    },
    "Recurrence": {
        "system": "http://terminology.hl7.org/CodeSystem/condition-clinical",
        "code": "recurrence",
        "display": "Recurrence"
    },
    "Relapse": {
        "system": "http://terminology.hl7.org/CodeSystem/condition-clinical",
        "code": "relapse",
        "display": "Relapse"
    },
    "Inactive": {
        "system": "http://terminology.hl7.org/CodeSystem/condition-clinical",
        "code": "inactive",
        "display": "Inactive"
    },
    "Remission": {
        "system": "http://terminology.hl7.org/CodeSystem/condition-clinical",
        "code": "remission",
        "display": "Remission"
    },
    "Resolved": {
        "system": "http://terminology.hl7.org/CodeSystem/condition-clinical",
        "code": "resolved",
        "display": "Resolved"
    }
}

def random_with_N_digits(n):
    range_start = 10**(n-1)
    range_end = (10**n)-1
    return randint(range_start, range_end)

# Custom serializer function
def custom_serializer(obj):
    if isinstance(obj, Decimal):
        return float(obj)  # or str(obj) if you want to represent as string
    raise TypeError(f"Object of type {obj.__class__.__name__} is not JSON serializable")

#Definining Condition_IDFHIR Class
class Condition_IDFHIR:
    def __init__(self):
        self.Condition = Condition(
            # Initialize necessary Procedure attributes here
                clinicalStatus= Condition_ClinicalStatus,
                identifier=([Identifier(system=f"http://sys-ids.kemkes.go.id/condition/{Org_Id}", value=identifier_value)]),
                code= CodeableConcept(coding=[Condition_coding]),
                category = [CodeableConcept(coding=[category_coding])],
                subject = Subject,
                encounter = Encounter,
                recordedDate= DateTime,
                recorder= Performer
        )
        return None
    
    def to_dict(self):
        """Convert the Procedure resource to a dictionary."""
        return self.Condition.dict() #returns dictionary (for use in PostmanCollection Builder)

    def to_json(self):
        """Convert the Procedure resource to a JSON string."""
        return self.Condition.json(indent=4)# returns beautified JSON

# Insert used parameters
Encounter_uuid = "{{Encounter_uuid}}" # @param {type:"string"}
Org_Id= '{{Org_id}}' # @param {type: 'string'}
Condition_MRID= '202401A000' + str(random_with_N_digits(4))
Condition_Status = "Active" # @param ["Active","Recurrence","Relapse","Inactive","Remission","Resolved"]{astype:"string"} 
Condition_Category= "Problem List Item" # @param ["Problem List Item","Encounter Diagnosis"]{astype:"string"}
Condition_CodeCodingSystem = "http://snomed.info/sct" # @param {type:"string"}
Condition_CodeCodingCode = "25064002" # @param {type:"string"}
Condition_CodeCodingDisplay = "Headache" # @param {type:"string"}
Patient_Id = "{{Patient_Id}}" # @param {type:"string"}
Patient_PMID= "P02029385699" # @param {type:"string"}
Patient_String = "{{Patient_Name}}" # @param {type:"string"}
Practitioner_Id = "{{Practitioner_Id}}" # @param {type:"string"}
Practitioner_String = "{{Practitioner_Name}}" # @param {type:"string"}
Location_Id ="1a6a5221-74ad-488f-a37a-d30657719e56" # @param {type:"string"}
Location_String = "Ruang Tumbuh Kembang, Departemen Kesehatan Keluarga" # @param {type:"string"}
DateTime_Year = 2023
DateTime_Month= 9
DateTime_Day= 2
DateTime_Hour= 0
DateTime_Min= 0

identifier_system= b'http://sys-ids.kemkes.go.id/Procedure/{{Org_id}}'
identifier_value= f"{Condition_MRID}"

#Operation based on Procedure input choices

DateTime= datetime(
    year= DateTime_Year, 
    month= DateTime_Month, 
    day= DateTime_Day, 
    hour= DateTime_Hour, 
    minute= DateTime_Min,
    tzinfo= timezone.utc
    )

# Set up the patient reference
Subject = f'Patient/{Patient_Id}'
Subject = Reference(reference=Subject, display= Patient_String)

#Set up the Encounter reference
Encounter = 'Encounter/{Encounter_uuid}'
Encounter = Encounter.format(Encounter_uuid=Encounter_uuid)
DateTime_string= DateTime.strftime("%d %b %Y")
Encounter_display= f"Kunjungan {Patient_String} pada tanggal {DateTime_string}"
Encounter = Reference(reference=Encounter, display= Encounter_display)

Performer= f'Practitioner/{Practitioner_Id}'
Performer= Reference(reference= Performer, display= Practitioner_String)

ClinicalStatus_Coding= Coding(
        system= ConditionClinicalStatus_Dict[Condition_Status]["system"],
        code= ConditionClinicalStatus_Dict[Condition_Status]["code"],
        display= ConditionClinicalStatus_Dict[Condition_Status]["display"]
    )

Condition_ClinicalStatus= CodeableConcept(
    coding= [ClinicalStatus_Coding]
)


# Set up category
category_coding = Coding(
    system= ConditionCategory_Dict[Condition_Category]["system"], 
    code=ConditionCategory_Dict[Condition_Category]["code"], 
    display=ConditionCategory_Dict[Condition_Category]["display"]
    )

# Set up code
Condition_coding = Coding(
    system=Condition_CodeCodingSystem, 
    code=Condition_CodeCodingCode, 
    display=Condition_CodeCodingDisplay
    )


#Create Condition Instance
#my_obs = Condition.construct()
#my_obs.clinicalStatus= Condition_ClinicalStatus
#my_obs.code= CodeableConcept(coding=[Condition_coding]),
#my_obs.category = CodeableConcept(coding=[category_coding]),
##my_obs.subject = Reference(reference=Subject, display= Patient_String),
#my_obs.encounter = Encounter,
#my_obs.onsetDateTime= DateTime,
#my_obs.recorder= Reference(reference= Practitioner_Id, display= Practitioner_String)
#converting to JSON and printing
#pretty_json = my_obs.to_json()
#print(pretty_json)#

resource= Condition_IDFHIR()
print(resource.to_json())

{
    "resourceType": "Condition",
    "identifier": [
        {
            "system": "http://sys-ids.kemkes.go.id/condition/{{Org_id}}",
            "value": "202401A0004988"
        }
    ],
    "clinicalStatus": {
        "coding": [
            {
                "system": "http://terminology.hl7.org/CodeSystem/condition-clinical",
                "code": "active",
                "display": "Active"
            }
        ]
    },
    "category": [
        {
            "coding": [
                {
                    "system": "http://terminology.hl7.org/CodeSystem/condition-category",
                    "code": "problem-list-item",
                    "display": "Problem List Item"
                }
            ]
        }
    ],
    "code": {
        "coding": [
            {
                "system": "http://snomed.info/sct",
                "code": "25064002",
                "display": "Headache"
            }
        ]
    },
    "subject": {
        "reference": "Patient/